In [1]:
import tensorflow as tf
from tensorflow.keras import Model, layers, Input

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


In [2]:
class eye_model(layers.Layer):
  def __init__(self, name='Eye-model'):
    super(eye_model, self).__init__()

    self.conv1 = layers.Conv2D(32, kernel_size=7, strides=2, padding='valid')
    self.conv2 = layers.Conv2D(64, kernel_size=5, strides=2, padding='valid')
    self.conv3 = layers.Conv2D(128, kernel_size=3, strides=1, padding='valid')
    self.bn = layers.BatchNormalization(axis = 1, momentum=0.9)
    self.leakyrelu = layers.LeakyReLU(alpha=0.01)
    self.avgpool = layers.AveragePooling2D(pool_size=2)
    self.dropout = layers.Dropout(rate=0.02)
    

  def call(self, input_image):
    x = self.conv1(input_image)
    x = self.bn(x)
    x = self.leakyrelu(x)
    x = self.avgpool(x)
    x = self.dropout(x)
    
    x = self.conv2(x)
    x = self.bn(x)
    x = self.leakyrelu(x)
    x = self.avgpool(x)
    x = self.dropout(x)
    
    x = self.conv3(x)
    x = self.bn(x)
    x = self.leakyrelu(x)
    x = self.avgpool(x)
    x = self.dropout(x)
    
    return x

In [3]:
class landmark_model(layers.Layer):
  def __init__(self, name='Landmark-model'):
    super(landmark_model, self).__init__()

    self.dense1 = layers.Dense(128)
    self.dense2 = layers.Dense(16)
    self.dense3 = layers.Dense(16)
    self.bn = layers.BatchNormalization(momentum=0.9)
    self.relu = layers.ReLU()

  def call(self, input_kps):
    x = self.dense1(input_kps)
    x = self.bn(x)
    x = self.relu(x)
    
    x = self.dense2(x)
    x = self.bn(x)
    x = self.relu(x)
    
    x = self.dense3(x)
    x = self.bn(x)
    x = self.relu(x)   
    
    return x

In [4]:
class gazetrack_model(Model):
  def __init__(self, name='Gazetrack-model'):
    super(gazetrack_model, self).__init__()

    self.eye_model = eye_model()
    self.lmModel = landmark_model()
    
    self.dense1 = layers.Dense(8)
    self.dense2 = layers.Dense(4)
    self.dense3 = layers.Dense(2)
    
    self.bn = layers.BatchNormalization(momentum=0.9)
    self.dropout = layers.Dropout(rate=0.12)
    self.relu = layers.ReLU()

    

  def call(self, [leftEye, rightEye, lms]):
    l_eye_feat = tf.reshape(self.eye_model(leftEye), (3, 128*128))
    r_eye_feat = tf.reshape(self.eye_model(rightEye), (3, 128*128))
    
    lm_feat = self.lmModel(lms)
    
    combined_feat = tf.concat((l_eye_feat, r_eye_feat, lm_feat),1)
    
    x = self.dense1(combined_feat)
    x = self.bn(x)
    x = self.dropout(x)
    x = self.relu(x)
    
    x = self.dense2(x)
    x = self.bn(x)
    x = self.relu(x)
    
    x = self.dense3(x)
    
    return x

SyntaxError: invalid syntax (<ipython-input-4-ff02bfce5e50>, line 18)

In [ ]:
in_leye = Input(shape(128,128,3))